# EDA and Structural Cleaning

In [ ]:
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt

## Making lite version for training

In [ ]:
RAW_DATA_PATH = '../data/raw/lending-club-full.csv'
LITE_DATA_PATH = '../data/processed/lending-club-lite.csv'

In [ ]:
# Cut the dataset to include essential columns only
cols_to_keep = [
    'loan_amnt', 'term', 'int_rate', 'installment', 'grade', 'sub_grade',
    'emp_length', 'home_ownership', 'annual_inc', 'verification_status',
    'issue_d', 'loan_status', 'purpose', 'dti', 'earliest_cr_line',
    'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc',
    'initial_list_status', 'application_type', 'mort_acc', 'pub_rec_bankruptcies'
]

df = pd.read_csv(RAW_DATA_PATH, usecols=cols_to_keep)

print(f'Raw dataset shape: {df.shape}')


In [ ]:
print(f'Distribution of target variable (count) - {df["loan_status"].value_counts()}')

In [ ]:
df = df[df['loan_status'].isin(['Fully Paid', 'Charged Off'])]

fully_paid = df[df['loan_status'] == 'Fully Paid']
charged_off = df[df['loan_status'] == 'Charged Off']

df = pd.concat([
    fully_paid.sample(n=4000, random_state=42),
    charged_off.sample(n=1000, random_state=42)
]).sample(frac=1, random_state=42).reset_index(drop=True)

print(f'Dataset shape: {df.shape}')

In [ ]:
# Save the lite dataset
os.makedirs(os.path.dirname(LITE_DATA_PATH), exist_ok=True)
df.to_csv(LITE_DATA_PATH, index=False)
print(f"Lite dataset saved to {LITE_DATA_PATH}")

## Checking the dataset

In [ ]:
df = pd.read_csv(LITE_DATA_PATH)

In [ ]:
print(f'Shape of the dataset: {df.shape}')

In [ ]:
print(f'Duplicates in the dataset: {df.duplicated().sum()}')

In [ ]:
print(df.dtypes.sort_values())

In [ ]:
print("Missing values per column:")
print(df.isnull().sum().sort_values(ascending=False))

Columns with missing values

`emp_length`: Employment length in years. Possible values are between 0 and 10 where 0 means less than one year and 10 means ten or more years.

`mort_acc`: Number of mortgage accounts.

`revol_util`: Revolving line utilization rate, or the amount of credit the borrower is using relative to all available revolving credit.

`pub_rec_bankruptcies`: Number of public record bankruptcies


In [ ]:
sns.heatmap(df.isnull(), cbar=False)

In [ ]:
print(f'Distribution of target variable (count) - {df["loan_status"].value_counts()}')
print(f'\nDistribution of target variable (%) - {df["loan_status"].value_counts(normalize=True) * 100}')

Why 80-20? Just to challenge myself but not too extreme as before that has 99-1. This is a part of learning.

In [ ]:
print(f'Negative numbers -> Income: {(df["annual_inc"] < 0).sum()} | Loan Amount: {(df["loan_amnt"] < 0).sum()} | DTI: {(df["dti"] < 0).sum()}')
print(f'Impossible values -> High DTI (>100): {(df["dti"] > 100).sum()}')

In [ ]:
obj_cols = df.select_dtypes(include='object').columns

print("Categories in column:\n")
for col in obj_cols:
    print(col)
    print(df[col].unique())
    print()

In [ ]:
df[obj_cols].nunique()

## Visualization

What insights to extract:
- loan_status vs features
- outliers (boxplot for numeric) -> may get capped
- distributions (barchart for categorical) -> minor category may get groupped into 'others'

In [ ]:
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()

if 'loan_status' in categorical_cols:
    categorical_cols.remove('loan_status')

In [ ]:
n_numeric = len(numeric_cols)
cols_per_row = 4

# ceiling division to ensure all columns fits
n_rows = (n_numeric + cols_per_row - 1) // cols_per_row

fig, axes = plt.subplots(n_rows, cols_per_row, figsize=(18, n_rows * 4))
axes = axes.flatten()

for idx, col in enumerate(numeric_cols):
    axes[idx].boxplot(df[col].dropna())
    axes[idx].set_title(f'{col}', fontsize=12, fontweight='bold')
    axes[idx].set_ylabel('Value')
    axes[idx].grid(True, alpha=0.3)

for idx in range(n_numeric, len(axes)):
    axes[idx].set_visible(False)
    
plt.tight_layout()
plt.show()

In [ ]:
n_categorical = len(categorical_cols)
cols_per_row = 3

# another ceiling division
n_rows = (n_categorical + cols_per_row - 1) // cols_per_row

fig, axes = plt.subplots(n_rows, cols_per_row, figsize=(18, n_rows * 4))
axes = axes.flatten()

for idx, col in enumerate(categorical_cols):
    value_counts = df[col].value_counts()
    axes[idx].bar(range(len(value_counts)), value_counts.values, color='steelblue')
    axes[idx].set_xticks(range(len(value_counts)))
    axes[idx].set_xticklabels(value_counts.index, rotation=45, ha='right')
    axes[idx].set_title(f'{col}', fontsize=12, fontweight='bold')
    axes[idx].set_ylabel('Count')
    axes[idx].grid(True, alpha=0.3)

for idx in range(n_categorical, len(axes)):
    axes[idx].set_visible(False)
    
plt.tight_layout()
plt.show()

In [ ]:
colors = {'Fully Paid': '#2ecc71', 'Charged Off': '#e74c3c'}
statuses = ['Fully Paid', 'Charged Off']

In [ ]:
n_numeric = len(numeric_cols)
cols_per_row = 4

n_rows = (n_numeric + cols_per_row - 1) // cols_per_row

fig, axes = plt.subplots(n_rows, cols_per_row, figsize=(20, n_rows * 4))
axes = axes.flatten()

for idx, col in enumerate(numeric_cols):
    data_to_plot = [df[df['loan_status'] == status][col].dropna() for status in statuses]
    bp = axes[idx].boxplot(data_to_plot, tick_labels=statuses, patch_artist=True)
    
    for patch, status in zip(bp['boxes'], statuses):
        patch.set_facecolor(colors[status])
        patch.set_alpha(0.7)
        
    axes[idx].set_title(f'{col}', fontsize=12, fontweight='bold')
    axes[idx].set_ylabel('Value')
    axes[idx].grid(True, alpha=0.3, axis='y')

for idx in range(n_numeric, len(axes)):
    axes[idx].set_visible(False)
    
plt.tight_layout()
plt.show()

In [ ]:
n_categorical = len(categorical_cols)
cols_per_row = 3

n_rows = (n_categorical + cols_per_row - 1) // cols_per_row

fig, axes = plt.subplots(n_rows, cols_per_row, figsize=(20, n_rows * 4))
axes = axes.flatten()

for idx, col in enumerate(categorical_cols):
    cross_tab = pd.crosstab(df[col], df['loan_status'])
    cross_tab.plot(kind='bar', ax=axes[idx], color=[colors['Fully Paid'], colors['Charged Off']], alpha=0.8)
    
    axes[idx].set_title(f'{col}', fontsize=12, fontweight='bold')
    axes[idx].set_ylabel('Count')
    axes[idx].set_xlabel('')
    axes[idx].legend(title='Loan Status', loc='upper right')
    axes[idx].grid(True, alpha=0.3, axis='y')
    axes[idx].tick_params(axis='x', rotation=45)

for idx in range(n_categorical, len(axes)):
    axes[idx].set_visible(False)
    
plt.tight_layout()
plt.show()

## Further Analysis: Missing Value

In [ ]:
missing_cols = ['emp_length', 'mort_acc', 'revol_util', 'pub_rec_bankruptcies']
n = len(df)

for c in missing_cols:
    m = df[c].isna().sum()
    
    print(f'\n=== {c} ===')
    print(f'Missing: {m} / {n} ({m/n:.2%})')
    print(df[c].describe(include='all'))
    

In [ ]:
for c in missing_cols:
    miss_rate_by_status = df.groupby('loan_status')[c].apply(lambda s: s.isna().mean()).sort_values(ascending=False)
    print(f'\n=== Missing rate by loan_status {c} ===')
    print(miss_rate_by_status)

In [ ]:
key_nums = ['loan_amnt', 'int_rate', 'annual_inc']

for c in missing_cols:
    flag = df[c].isna()
    
    print(f'\n=== {c}: compare w/ key numerics ===')
    
    for x in key_nums:
        if x not in df.columns:
            continue
        g_missing = df.loc[flag, x].dropna()
        g_present = df.loc[~flag, x].dropna()
        
        print(f'{x}: missing_mean={g_missing.mean():.3f} | present_mean={g_present.mean():.3f} | n_missing={len(g_missing)}')

In [ ]:
df.loc[df['emp_length'].isna(), ['loan_status','grade','sub_grade']].value_counts().head(20)

In [ ]:
credit_cols = ['total_acc','open_acc']
df.loc[df['mort_acc'].isna(), credit_cols].describe()
df.loc[~df['mort_acc'].isna(), credit_cols].describe()

## Cleaning the dataset

In [ ]:
# Store original shape for comparison
original_shape = df.shape[0]

# Create a copy for cleaning
df_cleaned = df.copy()

In [ ]:
# Handle missing values (row-independent operations)
print("Before cleaning:")
print("="*25)
print(f"Shape: {df.shape}")
print(f"Missing values:\n{df.isnull().sum()[df.isnull().sum() > 0]}\n")

df_cleaned = df_cleaned.dropna(subset=['revol_util', 'pub_rec_bankruptcies'])

df_cleaned['mort_acc_missing'] = df_cleaned['mort_acc'].isna().astype(int)
df_cleaned['emp_length_missing'] = df_cleaned['emp_length'].isna().astype(int)

print("After handling missing values:")
print("="*25)
print(f"Shape: {df_cleaned.shape}")
print(f"Missing values: {df_cleaned.isnull().sum().sum()}\n")

### Convert datetime

In [ ]:
# Convert the text to actual Datetime objects
# format='%b-%Y' tells Python that "Mar-2003" is Month-Year
df_cleaned['earliest_cr_line'] = pd.to_datetime(df_cleaned['earliest_cr_line'], format='%b-%Y')
df_cleaned['issue_d'] = pd.to_datetime(df_cleaned['issue_d'], format='%b-%Y')

# Pick a "Reference Date"
# Since this is historical data, we shouldn't use "today" (2026).
# We should use a date relevant to the dataset, like 2020 or the max date in the data.
# Let's assume the analysis is happening on Dec 31, 2020.
reference_date = pd.to_datetime('2018-12-31')

# Calculate the difference (Days -> Years)
# We divide by 365.25 to account for leap years
df_cleaned['credit_history_years'] = (reference_date - df_cleaned['earliest_cr_line']).dt.days / 365.25
df_cleaned['issue_years'] = (reference_date - df_cleaned['issue_d']).dt.days / 365.25

# Check the results
print(df_cleaned[['earliest_cr_line', 'credit_history_years']].head())
print(df_cleaned[['issue_d', 'issue_years']].head())

In [ ]:
# Drop the original date column because the model can't handle it
df_cleaned = df_cleaned.drop(columns=['earliest_cr_line', 'issue_d'])

# Verify it's gone
print(df_cleaned.info())

### Save Dataset

In [ ]:
# Save cleaned dataset
CLEANED_DATA_PATH = '../data/processed/lending-club-cleaned.csv'

df_cleaned.reset_index(drop=True, inplace=True)
df_cleaned.to_csv(CLEANED_DATA_PATH, index=False)

# Calculate actual rows removed
rows_removed = original_shape - df_cleaned.shape[0]
rows_removed_pct = (rows_removed / original_shape) * 100

print("CLEANING SUMMARY")
print("=" * 50)
print(f"Original dataset shape: {original_shape}")
print(f"Final dataset shape: {df_cleaned.shape[0]}")
print(f"Rows removed: {rows_removed} ({rows_removed_pct:.2f}%)")
print(f"\nCleaned dataset saved to: {CLEANED_DATA_PATH}")